In [79]:
import pandas as pd
import rosbag
import rospy

In [81]:
bag_path = "/home/stein/bagfiles/clock_always_on/test_right.bag"
topics_of_interest = ['/clock_sync']

bag = rosbag.Bag(bag_path)
messages = bag.read_messages(topics=topics_of_interest)

data_dict = {'index_time': [], 'common_time': []}

for topic, msg, t in messages:
    index_time = pd.Timestamp(t.to_sec(), unit='s')
    common_time = pd.Timestamp(msg.clock.secs + msg.clock.nsecs * 1e-9, unit='s')
    # print(index_time)
    # print(common_time)
    
    data_dict['index_time'].append(index_time)
    data_dict['common_time'].append(common_time)
    

df = pd.DataFrame(data_dict)
df['time_difference_seconds'] = (df['index_time'] - df['common_time']).dt.total_seconds()
mean_time_difference = df['time_difference_seconds'].mean()
print(f"Mean time difference: {mean_time_difference} seconds")

Mean time difference: 74.12775355379179 seconds


In [82]:
new_bag_path = "/home/stein/bagfiles/clock_always_on/right.bag"

with rosbag.Bag(new_bag_path, 'w') as new_bag:
    for topic, msg, timestamp in bag:
        modified_timestamp = timestamp - rospy.Duration.from_sec(mean_time_difference)
        new_bag.write(topic, msg, modified_timestamp)
